In [43]:
import os
import pickle
import pandas as pd
import torch

from torch.nn import Module, Sequential, Linear, Tanh

ARTIFACTS_PATH = '/usr/datalake/silver/stormevents/artifacts/igra_storm_event_autoencoder'
STATION_LIST = '/usr/datalake/silver/igra/doc/igra2-station-list.csv'
IGRA_SOURCE = '/usr/datalake/silver/igra/gph20s10k'
IGRA_DESTINATION = '/usr/datalake/silver/stormevents/igra_encoded'

In [44]:
def load_data(station_id: str) -> tuple[pd.DataFrame, pd.DataFrame]:
    # Load the dataset
    filename = f'{IGRA_SOURCE}/{station_id}-data-gph20s10k.csv'
    result = pd.read_csv(filename)

    # Remove irrelevant data
    result = result[result['hour'] == 12]
    result = result.drop(['id', 'hour', '0_gph',
                '1_gph', '2_gph', '3_gph', '4_gph', '5_gph',
                '6_gph', '7_gph', '8_gph', '9_gph', '10_gph',
                '11_gph', '12_gph', '13_gph', '14_gph', '15_gph',
                '16_gph', '17_gph', '18_gph', '19_gph', '20_gph'
                ], axis=1)
    result = result.reset_index(drop=True)

    index = result.iloc[:,:2]
    data = result.iloc[:,2:]

    return index, data

def load_scaler(station_id: str):
    with open(f'{ARTIFACTS_PATH}/{station_id}_scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)

    return scaler

def scale_data(scaler, sample: pd.DataFrame):
    result = torch.from_numpy(scaler.transform(sample)).float()
    return result

def load_model(station_id: str):
    model = AutoEncoder()
    model.load_state_dict(torch.load(f'{ARTIFACTS_PATH}/{station_id}_fnn.pt'))
    model.eval()

    return model

class AutoEncoder(Module):
    def __init__(self):
        super().__init__()
        
        self.encoder = Sequential(
            Linear(105, 80),
            Tanh(),
            Linear(80, 60),
            Tanh(),
            Linear(60, 40),
            Tanh(),
            Linear(40, 20)
        )

        self.decoder = Sequential(
            Linear(20, 40),
            Tanh(),
            Linear(40, 60),
            Tanh(),
            Linear(60, 80),
            Tanh(),
            Linear(80, 105)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)

        return decoded

In [45]:
os.makedirs(IGRA_DESTINATION, exist_ok=True)

In [46]:
df_brains = pd.read_csv(f'{ARTIFACTS_PATH}/brains_station_list.csv')
df_brains.head()

,id,latitude,longitude,elevation,state,name,fst_year,lst_year,nobs
0,BBM00078954,13.0716,-59.4922,56.6,NaN,GRANTLEY ADAMS,1965,2025,31817
1,BHM00078583,17.5333,-88.3000,5.0,NaN,BELIZE/PHILLIP GOLDSTON INTL.,1980,2025,21481
2,CJM00078384,19.2944,-81.3632,3.0,NaN,OWEN ROBERTS AIRPORT GRAND CAY,1956,2025,37597
3,COM00080001,12.5833,-81.7167,1.0,NaN,SAN ANDRES (ISLA)/SESQUICENTEN,1956,2025,27604
4,DRM00078486,18.4734,-69.8705,14.0,NaN,SANTO DOMINGO (78486-0),1962,2025,28418


In [47]:
for _, brain in df_brains.iterrows():
    station_id = brain['id']
    print(station_id)

    scaler = load_scaler(station_id)
    model = load_model(station_id)

    df_index, df_data = load_data(station_id)
    data = scale_data(scaler, df_data)
    
    # Encode the values
    logits = model.encoder(data).clone().detach().numpy()
    
    # Add the index back to the dataframe
    output = pd.DataFrame(logits)
    output = pd.concat([df_index, output], axis=1)

    output.to_csv(f'{IGRA_DESTINATION}/{station_id}_igra_encoded.csv', index=False)

BBM00078954
BHM00078583
CJM00078384
COM00080001
DRM00078486
JMM00078397
NNM00078866
RQM00078526
TDM00078970
UCM00078988
USM00070026
USM00070133
USM00070200
USM00070219
USM00070231
USM00070261
USM00070273
USM00070308
USM00070316
USM00070326
USM00070350
USM00070361
USM00070398
USM00072201
USM00072202
USM00072206
USM00072208
USM00072210
USM00072215
USM00072230
USM00072233
USM00072235
USM00072240
USM00072248
USM00072249
USM00072250
USM00072251
USM00072261
USM00072265
USM00072274
USM00072293
USM00072305
USM00072317
USM00072318
USM00072327
USM00072340
USM00072357
USM00072363
USM00072364
USM00072365
USM00072376
USM00072381
USM00072388
USM00072402
USM00072403
USM00072426
USM00072440
USM00072451
USM00072456
USM00072476
USM00072489
USM00072493
USM00072501
USM00072518
USM00072520
USM00072528
USM00072558
USM00072562
USM00072572
USM00072582
USM00072597
USM00072632
USM00072634
USM00072645
USM00072649
USM00072659
USM00072662
USM00072672
USM00072681
USM00072694
USM00072712
USM00072747
USM00072764
USM0

In [48]:
output.head()

,effective_date,day_num,0,1,2,3,4,5,6,7,...,10,11,12,13,14,15,16,17,18,19
0,2000-01-01,-1.0,-0.041933,-0.082313,-0.004124,0.132370,0.189991,-0.030891,0.087338,0.029286,...,0.057471,-0.044537,0.086868,-0.020065,-0.033794,-0.004521,0.095818,0.071102,-0.067359,-0.040800
1,2000-01-02,-1.0,0.038592,0.002951,0.110727,0.027365,0.137092,0.091406,0.102679,-0.123372,...,0.012125,-0.187060,-0.033473,0.023709,-0.021744,0.008940,0.010359,0.115556,-0.104181,-0.164971
2,2000-01-03,-1.0,-0.051094,-0.006552,0.029993,0.134049,0.106613,0.021681,0.047709,0.014681,...,-0.037441,-0.243535,-0.122872,0.017836,-0.172817,0.034610,-0.023638,0.120743,-0.194331,0.096304
3,2000-01-04,-1.0,0.087426,-0.022456,0.076684,0.048971,0.222363,-0.098132,0.046847,0.018535,...,0.046893,-0.129568,-0.054417,0.066768,-0.138138,0.024688,-0.137213,0.124050,-0.181782,0.053161
4,2000-01-05,-1.0,0.019022,-0.069662,0.037705,0.050153,0.191935,-0.131333,0.132230,0.042795,...,0.021116,-0.160682,-0.003729,0.158235,-0.135262,-0.035308,-0.126404,0.098189,-0.118489,0.031585
